### Note
前一版發現不同的submission之間差異太大，004版的auto-sklearn是直接做分類，其他版是回歸然後四捨五入取整。
本篇任務:
1. 用回歸算，四捨五入，看能不能做voting

In [1]:
import math
import warnings  
from tqdm import tqdm
import time
import pandas as pd
import os

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']

In [2]:
lis = ['data', 'vis', 'sub']
for i in lis:
    if os.path.isdir(i):
        print("%-5s目錄存在，不建立。"%i)
    else:
        os.makedirs(i)

data 目錄存在，不建立。
vis  目錄存在，不建立。
sub  目錄存在，不建立。


In [3]:
filename = './data/train.csv'
df = pd.read_csv(filename)

test_filename = './data/test.csv'
test = pd.read_csv(test_filename)

## Note
- y = LEVEL
- 文字: Station, County, Location
- 有nan

## 把Season 4的部分用眾數補起來

In [4]:
df_copy = df.copy()
nan_col = df_copy.columns[df_copy.isna().any()]

lis = df_copy['Station'].unique()
print(lis)
for i in lis:
    cond = df_copy['Station'] == i
    
    df_copy.loc[cond, nan_col] = df_copy.loc[cond, nan_col].fillna(0, axis = 0)
    df_copy.loc[cond, nan_col] = df_copy.loc[cond, nan_col].mode().values[0]

['E02' 'E03' 'E05' 'E06' 'E08' 'E09' 'HL02' 'HL03' 'HL05' 'HL06' 'HL08'
 'HL09' 'HL11' 'HL12' 'M01' 'M02' 'M04' 'M05' 'M07' 'M08' 'M10' 'M11'
 'M13' 'M14' 'M16' 'M17' 'M19' 'M20' 'M22' 'M23' 'N01' 'N02' 'N04' 'N05'
 'N07' 'N08' 'N10' 'N11' 'N13' 'N14' 'N16' 'N17' 'N19' 'N20' 'N22' 'N23'
 'SE03' 'SE04' 'SE06' 'SE07' 'SE09' 'SE10' 'SE12' 'SE13' 'SE15' 'SE16'
 'SE18' 'SE19' 'SE21' 'SE22' 'SE24' 'SW01' 'SW03' 'SW04' 'SW06' 'SW07'
 'SW09' 'SW10' 'SW12' 'SW13' 'SW15' 'SW16' 'SW18' 'SW19' 'SW21' 'SW22'
 'SW24' 'SW25' 'TT02' 'TT03']


## drop column

In [5]:
df_copy.pop('County')
df_copy.pop('Station')
df_copy.pop('Location')

0              大溪
1              大溪
2              大溪
3              大溪
4              頭城
          ...    
314            長光
315           白桑安
316    白桑安/長濱觀景平台
317    白桑安/長濱觀景平台
318            寜埔
Name: Location, Length: 319, dtype: object

In [6]:
df_train = df_copy.copy()

In [7]:
X = df_copy.copy()
y = X.pop('LEVEL')

In [8]:
import autosklearn.regression as auto_reg
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pickle
import numpy as np
# X, y = sklearn.datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, test_size = 0.3, random_state=12)

automl = auto_reg.AutoSklearnRegressor(
    time_left_for_this_task=1000,
    per_run_time_limit=200,
#     tmp_folder='./tmp/autosklearn_parallel_1_example_tmp',
#     output_folder='./tmp/autosklearn_parallel_1_example_out',
#     include_estimators = ['random_forest', 'libsvm_svc', ],
#     resampling_strategy='holdout',
#     resampling_strategy_arguments={'train_size': 0.8},
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5},
    ensemble_nbest = 30, 
    
    
    n_jobs=8,
    # Each one of the 4 jobs is allocated 3GB
    memory_limit=10240,
    seed=5,
)
automl.fit(X_train, y_train, dataset_name='ocean_001')

# Print statistics about the auto-sklearn run such as number of
# iterations, number of models failed with a time out.
print(automl.sprint_statistics())
y_hat = automl.predict(X_test)
y_hat = np.ceil(y_hat).astype('int')
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))



# save model
with open('./tmp/iris-classifier.pkl', 'wb') as f:
    pickle.dump(automl, f)

# load model
with open('./tmp/iris-classifier.pkl', 'rb') as f:
    loaded_classifier = pickle.load(f)

auto-sklearn results:
  Dataset name: ocean_001
  Metric: r2
  Best validation score: 0.424719
  Number of target algorithm runs: 110
  Number of successful target algorithm runs: 88
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 19
  Number of target algorithms that exceeded the memory limit: 0

Accuracy score 0.3333333333333333


In [28]:
test_filename = './data/test.csv'
df = pd.read_csv(test_filename)

tmp = pd.read_csv('data/submission.csv')
tmp.head()

,ID,LEVEL
0,E01_1,NaN
1,E01_2,NaN
2,E01_3,NaN
3,E01_4,NaN
4,E04_1,NaN


## 把地理因素補起來

In [29]:
df_copy = df.copy()
nan_col = df_copy.columns[df_copy.isna().any()]

lis = df_copy['Station'].unique()
print(lis)
for i in lis:
    cond = df_copy['Station'] == i
    
    df_copy.loc[cond, nan_col] = df_copy.loc[cond, nan_col].fillna(0, axis = 0)
    df_copy.loc[cond, nan_col] = df_copy.loc[cond, nan_col].mode().values[0]


['E01' 'E04' 'E07' 'HL01' 'HL04' 'HL07' 'HL10' 'LI10' 'M03' 'M06' 'M09'
 'M12' 'M15' 'M18' 'M21' 'M24' 'N03' 'N06' 'N09' 'N12' 'N15' 'N18' 'N21'
 'N24' 'SE05' 'SE08' 'SE11' 'SE14' 'SE17' 'SE20' 'SE23' 'SW02' 'SW05'
 'SW08' 'SW11' 'SW14' 'SW17' 'SW20' 'SW23' 'TT01' 'TT04']


# 做submmision

In [30]:
df_copy.pop('County')
df_copy.pop('Station')
df_copy.pop('Location')

0       石城
1       石城
2       石城
3       石城
4       永鎮
      ... 
158    八仙北
159    石雨傘
160    石雨傘
161    石雨傘
162    石雨傘
Name: Location, Length: 163, dtype: object

In [31]:
import pickle


with open('./tmp/iris-classifier.pkl', 'rb') as f:
    loaded_classifier = pickle.load(f)

In [32]:
import datetime
now_time = datetime.datetime.now()
now_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
now_time

'2021-06-05 17:09:20'

In [33]:
y_pred = loaded_classifier.predict(df_copy)
y_pred = np.ceil(y_pred).astype('int')

tmp = pd.read_csv('data/submission.csv')
tmp.loc[:, 'LEVEL'] = y_pred
tmp.to_csv('./sub/'+now_time+'_auto_sklearn.csv', index = False)

In [34]:
def new_report(test_report):
    lists = os.listdir(test_report)                                    #列出目錄的下所有文件和文件夾保存到lists
    print(list)
    lists.sort(key=lambda fn:os.path.getmtime(test_report + "/" + fn))#按時間排序
    file_new = os.path.join(test_report,lists[-1])                     #獲取最新的文件保存到file_new
    print(file_new)
    return file_new


In [35]:
if __name__=="__main__":
    test_report="sub"   #目錄地址
    new_report(test_report)


<class 'list'>
sub/2021-06-05 17:09:20_auto_sklearn.csv


In [40]:
a = pd.read_csv(new_report(test_report))['LEVEL']
b = pd.read_csv('sub/auto_sklearn (1).csv')['LEVEL']


print(np.array_equiv(a,b))
tmp = np.where(a!=b)[0]
print(len(tmp))


<class 'list'>
sub/2021-06-05 17:09:20_auto_sklearn.csv
False
133


兩個答案不重複的這麼多? 實際上差異的東西看起來會不會是排名上差1或2? 是否表示跟排名上有些許差異。

In [41]:
# test data
print(a[np.where(a!=b)[0]].values)
print(b[np.where(a!=b)[0]].values)

[5 6 6 6 6 5 6 5 5 4 5 4 5 6 4 5 5 4 4 4 5 5 5 4 5 6 4 7 7 7 7 8 7 7 7 7 7
 7 7 5 5 5 6 7 7 6 5 6 5 5 4 5 5 5 6 7 7 7 6 5 4 4 4 7 6 7 7 6 6 5 7 7 7 7
 7 6 5 5 5 4 3 4 4 5 4 5 4 4 4 4 4 6 6 6 7 6 5 7 6 7 6 6 6 6 6 6 5 4 4 4 6
 5 7 5 6 6 5 8 8 8 6 7 7 7 5 4 3 4 3 5 4 6 5]
[ 2  7  7  7  4  2  1  2  2  1  1  2  2  7  2  7  1  2  2  2  7  1  2  2
  7  7  2  6 10  6  6 10  6 10  6  6  6  6  6  2  2  2 10  6 10  2  2 10
  2  2  2  2  2  2  2  9  9  9  8  2  2  2  2  9  2  8  8  9  2  2  8  8
  8  8  8  2  2  2  2  2  2  2  2  7  1  1  1  2  2  2  2  5  7  5  5  1
  1  3  5  3  3  5  5  5  7  5  3  3  3  3  7  2  9  2  5  2  2  6  6  9
  5  6  6  6  2  1  1  2  2  2  2  7  2]
